In [1]:
!nvcc --version
!nvidia-smi
import torch
torch.__version__

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Thu May 16 13:42:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  7

'2.2.1+cu121'

In [2]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git@5aeb252b194b93dc2879b4ac34bc51a31b5aee13'

  Cloning https://github.com/facebookresearch/detectron2.git (to revision 5aeb252b194b93dc2879b4ac34bc51a31b5aee13) to /tmp/pip-req-build-m9fizpv1
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-m9fizpv1
  Running command git rev-parse -q --verify 'sha^5aeb252b194b93dc2879b4ac34bc51a31b5aee13'
  Running command git fetch -q https://github.com/facebookresearch/detectron2.git 5aeb252b194b93dc2879b4ac34bc51a31b5aee13
  Running command git checkout -q 5aeb252b194b93dc2879b4ac34bc51a31b5aee13
  Resolved https://github.com/facebookresearch/detectron2.git to commit 5aeb252b194b93dc2879b4ac34bc51a31b5aee13
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 787.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 8.2 MB/s eta

In [3]:
%cd /content/
!git clone https://github.com/DomMcOyle/TACO-expl.git
%cd /content/TACO-expl
!git checkout maskdino
!git pull origin maskdino
%cd /content/TACO-expl/MaskDINO
!pip install -r requirements.txt
%cd /content/TACO-expl/MaskDINO/maskdino/modeling/pixel_decoder/ops
!sh make.sh

/content
Cloning into 'TACO-expl'...
remote: Enumerating objects: 2335, done.
remote: Counting objects: 100% (2335/2335), done.
remote: Compressing objects: 100% (1123/1123), done.
remote: Total 2335 (delta 1224), reused 2255 (delta 1173), pack-reused 0
Receiving objects: 100% (2335/2335), 130.63 MiB | 16.22 MiB/s, done.
Resolving deltas: 100% (1224/1224), done.
Updating files: 100% (28/28), done.
/content/TACO-expl
Branch 'maskdino' set up to track remote branch 'maskdino' from 'origin'.
Switched to a new branch 'maskdino'
From https://github.com/DomMcOyle/TACO-expl
 * branch            maskdino   -> FETCH_HEAD
Already up to date.
/content/TACO-expl/MaskDINO
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 1.6 MB/s eta 0:00:00
/content/TACO-expl/MaskDINO/maskdino/modeling/pixel_decoder/ops
running build
running build_py
creating build
creating build/lib.linux-x86_64-cpython-310
creating build/lib.linux-x86_64-cpython-310/functions
copying functions/ms_deform_attn_func.py -> 

In [4]:
from google.colab import drive
drive.mount("/content/MyDrive/", force_remount = True)

Mounted at /content/MyDrive/


In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.2 ; cuda:  cu121
detectron2: 0.6


In [2]:
%cd /content/
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

/content


In [3]:
import sys
sys.path.append("/content/TACO-expl/MaskDINO")
!cd /content/TACO-expl/MaskDINO/
cfg_biggest_path = '/content/TACO-expl/MaskDINO/configs/coco/instance-segmentation/swin/maskdino_R50_bs16_50ep_4s_dowsample1_2048.yaml'
model_biggest_weights_url = 'https://github.com/IDEA-Research/detrex-storage/releases/download/maskdino-v0.1.0/maskdino_swinl_50ep_300q_hid2048_3sd1_instance_maskenhanced_mask52.3ap_box59.0ap.pth'

cfg_smallest_path = '/content/TACO-expl/MaskDINO/configs/coco/instance-segmentation/maskdino_R50_bs16_50ep_3s.yaml'
model_smallest_weight_url = 'https://github.com/IDEA-Research/detrex-storage/releases/download/maskdino-v0.1.0/maskdino_r50_50ep_300q_hid1024_3sd1_instance_maskenhanced_mask46.1ap_box51.5ap.pth'

In [4]:
sys.path.append('/content/TACO-expl/MaskDINO/maskdino/')
from config import add_maskdino_config
%cd /content/TACO-expl/MaskDINO/maskdino/
from detectron2.projects.deeplab import add_deeplab_config
from maskdino import MaskDINO

def setup_cfg(cfg_path, weights_url):
    # load config from file and command-line arguments
    cfg = get_cfg()
    add_deeplab_config(cfg)
    add_maskdino_config(cfg)
    cfg.merge_from_file(cfg_path)
    #cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    #cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.5
    cfg.MODEL.WEIGHTS = weights_url
    cfg.freeze()
    return cfg

cfg_alt_dino = setup_cfg(cfg_smallest_path, model_smallest_weight_url)
cfg_modified = cfg_alt_dino.copy()
#print(cfg_modified["MODEL"])

/content/TACO-expl/MaskDINO/maskdino


/content/TACO-expl/MaskDINO/maskdino/modeling/criterion.py:346: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self.dn is not "no" and mask_dict is not None:


In [5]:
%cd /content/TACO-expl/
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

import os.path
import json
import argparse
import numpy as np
import random
import datetime as dt
import copy
import torch.nn as nn
from sklearn.model_selection import train_test_split
from pathlib import Path

import torch
import math
import torch.utils.data
from torch.utils.data import DataLoader
from pycocotools import mask as coco_mask


from torchvision.ops import RoIAlign
from HDDETR.datasets.torchvision_datasets.coco import CocoDetection as TvCocoDetection # from torchvision.datasets import CocoDetection as TvCocoDetection
                                                                                       # the right above import import does not work bc of attribute definitions
import HDDETR.datasets.transforms as T #from torchvision import transforms as T # this import does not contain the randomselect method
import HDDETR.util.misc as mutils

/content/TACO-expl


In [6]:
DEFAULT_RANDOM_SEED = 42
# basic random seed
def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
# torch random seed
def seedTorch(seed=DEFAULT_RANDOM_SEED):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
# combine
def seedEverything(seed=DEFAULT_RANDOM_SEED):
    seedBasic(seed)
    seedTorch(seed)

seedEverything()

In [7]:
keep_categories = ["Bottle", "Bottle cap", "Can", "Cigarette", "Cup",
                   "Lid", "Plastic bag & wrapper", "Pop tab", "Straw"]

def create_map(original, keep_supercategories):
  class_map = {}
  for cat in original:
    if cat["supercategory"] in keep_supercategories:
      class_map[cat["name"]] = cat["supercategory"]
    else:
      class_map[cat["name"]] = "Other"
  return class_map

def replace_dataset_classes(dataset, class_map):
      """ Replaces classes of dataset based on a dictionary"""
      class_new_names = list(set(class_map.values()))
      class_new_names.sort()
      class_originals = copy.deepcopy(dataset['categories'])
      dataset['categories'] = []
      class_ids_map = {}  # map from old id to new id

      # Assign background id 0
      has_background = False
      if 'Background' in class_new_names:
          if class_new_names.index('Background') != 0:
              class_new_names.remove('Background')
              class_new_names.insert(0, 'Background')
          has_background = True

      # Replace categories
      for id_new, class_new_name in enumerate(class_new_names):
          # Make sure id:0 is reserved for background
          id_rectified = id_new
          if not has_background:
              id_rectified += 1

          category = {
              'supercategory': '',
              'id': id_rectified,  # Background has id=0
              'name': class_new_name,
          }
          dataset['categories'].append(category)
          # Map class names
          for class_original in class_originals:
              if class_map[class_original['name']] == class_new_name:
                  class_ids_map[class_original['id']] = id_rectified

      # Update annotations category id tag
      for ann in dataset['annotations']:
          ann['category_id'] = class_ids_map[ann['category_id']]

In [8]:
with open("/content/TACO-expl/data/annotations.json", "r") as f: #"/content/TACO/data/annotations.json"
    dataset = json.loads(f.read())

class_map = create_map(dataset["categories"], keep_categories)
replace_dataset_classes(dataset, class_map)
dataset["categories"]

[{'supercategory': '', 'id': 1, 'name': 'Bottle'},
 {'supercategory': '', 'id': 2, 'name': 'Bottle cap'},
 {'supercategory': '', 'id': 3, 'name': 'Can'},
 {'supercategory': '', 'id': 4, 'name': 'Cigarette'},
 {'supercategory': '', 'id': 5, 'name': 'Cup'},
 {'supercategory': '', 'id': 6, 'name': 'Lid'},
 {'supercategory': '', 'id': 7, 'name': 'Other'},
 {'supercategory': '', 'id': 8, 'name': 'Plastic bag & wrapper'},
 {'supercategory': '', 'id': 9, 'name': 'Pop tab'},
 {'supercategory': '', 'id': 10, 'name': 'Straw'}]

In [9]:
classes = [elem["name"] for elem in dataset["categories"]]
print(classes)

['Bottle', 'Bottle cap', 'Can', 'Cigarette', 'Cup', 'Lid', 'Other', 'Plastic bag & wrapper', 'Pop tab', 'Straw']


In [10]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog

help(register_coco_instances)

Help on function register_coco_instances in module detectron2.data.datasets.coco:

register_coco_instances(name, metadata, json_file, image_root)
    Register a dataset in COCO's json annotation format for
    instance detection, instance segmentation and keypoint detection.
    (i.e., Type 1 and 2 in http://cocodataset.org/#format-data.
    `instances*.json` and `person_keypoints*.json` in the dataset).
    
    This is an example of how to register a new dataset.
    You can do something similar to this function, to register new datasets.
    
    Args:
        name (str): the name that identifies a dataset, e.g. "coco_2014_train".
        metadata (dict): extra metadata associated with this dataset.  You can
            leave it as an empty dict.
        json_file (str): path to the json instance annotation file.
        image_root (str or path-like): directory which contains all the images.



In [11]:
train_annotation_file = '/content/TACO-expl/data/annotations_off_0_train.json'
val_annotation_file = '/content/TACO-expl/data/annotations_off_0_val.json'

img_dir = '/content/MyDrive/MyDrive/official/'

register_coco_instances("TACO_train", {}, train_annotation_file, img_dir)
MetadataCatalog.get("TACO_train").set(thing_classes = classes)
dataset_dicts_train = DatasetCatalog.get("TACO_train")

register_coco_instances("TACO_val", {}, val_annotation_file, img_dir)
MetadataCatalog.get("TACO_val").set(thing_classes = classes)
dataset_dicts_val = DatasetCatalog.get("TACO_val")

[05/16 13:52:54 d2.data.datasets.coco]: Loaded 1200 images in COCO format from /content/TACO-expl/data/annotations_off_0_train.json
[05/16 13:52:55 d2.data.datasets.coco]: Loaded 150 images in COCO format from /content/TACO-expl/data/annotations_off_0_val.json


In [12]:
import json

with open(val_annotation_file, "r") as f:
  val_annotations = json.load(f)

print(val_annotations.keys())

dict_keys(['info', 'images', 'annotations', 'scene_annotations', 'licenses', 'categories', 'scene_categories'])


In [13]:
first_img = val_annotations["images"][0]
first_ann = val_annotations["annotations"][0]

print(first_img)
print(first_ann)

{'id': 899, 'width': 3264, 'height': 2448, 'file_name': 'batch_4/000023.JPG', 'license': None, 'flickr_url': 'https://farm66.staticflickr.com/65535/47803892332_5218b74150_o.png', 'coco_url': None, 'date_captured': None, 'flickr_640_url': 'https://farm66.staticflickr.com/65535/47803892332_f552fb65c7_z.jpg'}
{'id': 47, 'image_id': 15, 'category_id': 1, 'segmentation': [[249.0, 1458.0, 236.0, 1360.0, 222.0, 1247.0, 150.0, 784.0, 150.0, 747.0, 157.0, 712.0, 172.0, 675.0, 184.0, 656.0, 205.0, 632.0, 238.0, 603.0, 263.0, 589.0, 299.0, 575.0, 312.0, 570.0, 322.0, 568.0, 324.0, 567.0, 323.0, 538.0, 319.0, 497.0, 314.0, 432.0, 307.0, 362.0, 307.0, 354.0, 313.0, 346.0, 317.0, 343.0, 328.0, 338.0, 345.0, 333.0, 355.0, 330.0, 363.0, 328.0, 371.0, 334.0, 386.0, 332.0, 404.0, 330.0, 421.0, 330.0, 450.0, 332.0, 476.0, 335.0, 486.0, 338.0, 489.0, 337.0, 493.0, 335.0, 501.0, 337.0, 509.0, 341.0, 518.0, 346.0, 527.0, 351.0, 531.0, 359.0, 531.0, 369.0, 532.0, 400.0, 533.0, 429.0, 538.0, 526.0, 541.0, 566

In [14]:
!mkdir /content/resized_validation/

In [18]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import itertools

def visualize_mask(mask):
    plt.figure(figsize=(8, 8))
    plt.imshow(mask)
    plt.axis('off')
    plt.show()

def flatten(aList):
  t = []
  for i in aList:
    if not isinstance(i, list):
      t.append(i)
    else:
      t.extend(flatten(i))
  return t

def mask_to_polygon(mask):
    # Find contours
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    polygons = []
    for contour in contours:
        # Convert polygon to list of coordinates
        polygon = contour.squeeze().astype(float).tolist()
        polygons.append(polygon)
    #polygons = flatten(polygons)
    return polygons

def visualize_polygons(polygons):
    plt.figure(figsize=(8, 8))
    for polygon in polygons:
        polygon = np.array(polygon)
        plt.plot(polygon[:, 0], polygon[:, 1], color='red', linewidth=2)
    plt.axis('off')
    plt.show()

"""
def group_coords_poly(ungrouped_poly):
  to_return = []
  for inner_list in ungrouped_poly:
    to_append = [[inner_list[i], inner_list[i+1]] for i in range(0, len(inner_list), 2)]
    to_return.append(to_append)
  return to_return
"""
def group_coords_poly(ungrouped_poly):
  ung_poly = ungrouped_poly.copy()
  ung_poly = flatten(ung_poly)
  to_return = [[[ung_poly[i], ung_poly[i+1]] for i in range(0, len(ung_poly), 2)]]
  return to_return

def ungroup_coords_poly(grouped_poly):
  #to_return = [[elem for pair_inner_list in inner_list for elem in pair_inner_list] for inner_list in grouped_poly]
  to_return = [flatten(grouped_poly)]
  return to_return

#some_ann = [[2152.0, 1374.0, 2206.0, 1375.0, 2239.0, 1366.0], [2262.0, 1358.0, 2275.0, 1342.0, 2270.0, 1323.0, 2259.0, 1312.0, 2249.0, 1308.0, 2251.0, 1291.0, 2236.0, 1270.0, 2218.0, 1259.0, 2191.0, 1254.0, 2149.0, 1256.0, 2125.0, 1259.0, 2109.0, 1272.0, 2098.0, 1285.0, 2093.0, 1294.0, 2090.0, 1307.0, 2080.0, 1314.0, 2069.0, 1324.0, 2073.0, 1347.0, 2095.0, 1363.0, 2122.0, 1370.0, 2152.0, 1374.0]]
#to_print = group_coords_poly(some_ann)
#print(to_print)
#print(ungroup_coords_poly(to_print))

In [19]:
from detectron2.data.transforms.augmentation_impl import ResizeShortestEdge
from detectron2.data.transforms import ResizeTransform
from PIL import Image
from pycocotools import mask as coco_mask
from skimage import measure
#import pycocotools.mask as mask_util

resize_obj_func = ResizeShortestEdge(800,
                                  1333,
                                  sample_style = "choice")

res_imgs_json_path = "/content/resized_taco_validation.json"
where_save = "/content/resized_validation/"
res_imgs_list = []
res_annotation_list = []

for img_meta in val_annotations["images"]:
  res_img_dict = img_meta.copy()
  id = img_meta["id"]
  height = img_meta["height"]
  width = img_meta["width"]
  img_path = img_meta["file_name"]
  new_h, new_w = ResizeShortestEdge.get_output_shape(height, width, 800, 1333)
  res_img_dict.update({"height": height, "width": width})
  res_imgs_list.append(res_img_dict)

  trans_func = ResizeTransform(height, width, new_h, new_w, Image.BILINEAR)
  more_segs = False
  for annotation in val_annotations["annotations"]:
    if annotation["image_id"] == id:
      res_segm_dict = annotation.copy()
      rles = coco_mask.frPyObjects(annotation["segmentation"], height, width)
      mask = coco_mask.decode(rles)
      if len(mask.shape) > 2:
        mask = np.sum(mask, axis = -1, dtype = mask.dtype)

      #print(mask.shape)
      #visualize_mask(mask)
      res_mask = trans_func.apply_segmentation(mask)
      #print(res_mask.dtype)
      #visualize_mask(res_mask)
      res_mask = np.asfortranarray(np.squeeze(res_mask))


      if len(res_mask.shape) > 2:
        res_mask = np.sum(res_mask, axis = -1)

      encoded_ground_truth = coco_mask.encode(res_mask)
      ground_truth_area = coco_mask.area(encoded_ground_truth)
      ground_truth_bounding_box = coco_mask.toBbox(encoded_ground_truth)
      res_mask_poly = mask_to_polygon(res_mask)
      print("_____Original Segmentation_____")
      print(annotation["segmentation"])
      #visualize_polygons(np.array(group_coords_poly(annotation["segmentation"])))

      #visualize_polygons(np.array(group_coords_poly([res_mask_poly])))
      ungrouped_mask = ungroup_coords_poly(res_mask_poly)
      print("_____Resized Segmentation_____")
      print(ungrouped_mask)
      res_segm_dict.update({"segmentation": ungrouped_mask,
                            "area": ground_truth_area,
                            "bbox": ground_truth_bounding_box})
      res_annotation_list.append(res_segm_dict)

_____Original Segmentation_____
[[2152.0, 1374.0, 2206.0, 1375.0, 2239.0, 1366.0, 2262.0, 1358.0, 2275.0, 1342.0, 2270.0, 1323.0, 2259.0, 1312.0, 2249.0, 1308.0, 2251.0, 1291.0, 2236.0, 1270.0, 2218.0, 1259.0, 2191.0, 1254.0, 2149.0, 1256.0, 2125.0, 1259.0, 2109.0, 1272.0, 2098.0, 1285.0, 2093.0, 1294.0, 2090.0, 1307.0, 2080.0, 1314.0, 2069.0, 1324.0, 2073.0, 1347.0, 2095.0, 1363.0, 2122.0, 1370.0, 2152.0, 1374.0]]
_____Resized Segmentation_____
[[701.0, 410.0, 700.0, 411.0, 694.0, 411.0, 686.0, 419.0, 686.0, 420.0, 685.0, 421.0, 685.0, 422.0, 684.0, 423.0, 684.0, 425.0, 683.0, 426.0, 683.0, 427.0, 682.0, 428.0, 681.0, 428.0, 677.0, 432.0, 676.0, 432.0, 676.0, 433.0, 677.0, 434.0, 677.0, 438.0, 678.0, 439.0, 678.0, 440.0, 679.0, 441.0, 680.0, 441.0, 681.0, 442.0, 682.0, 442.0, 684.0, 444.0, 685.0, 444.0, 686.0, 445.0, 688.0, 445.0, 689.0, 446.0, 692.0, 446.0, 693.0, 447.0, 699.0, 447.0, 700.0, 448.0, 723.0, 448.0, 724.0, 447.0, 727.0, 447.0, 728.0, 446.0, 730.0, 446.0, 731.0, 445.0, 73

In [20]:
print(len(res_imgs_list))

150


In [21]:
print(len(res_annotation_list))

410


##The annotations have been resized, now this must be serialized and used at validation time.
#N.B.: the config file must be loaded in the appropriate folder

In [ ]:
print(res_imgs_list)
print(res_annotation_list)

In [ ]:
dataset_dicts_val[0]["annotations"][0].keys()

dict_keys(['iscrowd', 'bbox', 'category_id', 'segmentation', 'bbox_mode'])

In [ ]:
"""
    def __call__(self, dataset_dict):
        #dataset_dict = utils.copy_instance(dataset_dict)

        image = utils.read_image(dataset_dict["file_name"], format="RGB")
        resize_obj = ResizeShortestEdge(self.cfg.INPUT.MIN_SIZE_TEST,
                                        self.cfg.INPUT.MAX_SIZE_TEST,
                                        sample_style = "choice")
        augs = AugmentationList([resize_obj])
        bboxes = [dataset_dict["annotations"][elem]["bbox"]
                  for elem in range(0, len(dataset_dict["annotations"]))]
        sem_segs = [dataset_dict["annotations"][elem]["segmentation"]
                    for elem in range(0, len(dataset_dict["annotations"]))]
        input_to_aug = AugInput(image,
                                boxes = bboxes,
                                sem_seg = sem_segs)
        to_return = augs(input_to_aug)
        #resize_img_func = resize_obj.get_transform(image)
        #dataset_dict["image"] = resize_img_func.apply_image(image)
        #dataset_dict["image"] = resize_img.apply(dataset_dict["image"])
        return to_return
    """

'\n    def __call__(self, dataset_dict):\n        #dataset_dict = utils.copy_instance(dataset_dict)\n        \n        image = utils.read_image(dataset_dict["file_name"], format="RGB")\n        resize_obj = ResizeShortestEdge(self.cfg.INPUT.MIN_SIZE_TEST, \n                                        self.cfg.INPUT.MAX_SIZE_TEST,\n                                        sample_style = "choice")\n        augs = AugmentationList([resize_obj])\n        bboxes = [dataset_dict["annotations"][elem]["bbox"] \n                  for elem in range(0, len(dataset_dict["annotations"]))]\n        sem_segs = [dataset_dict["annotations"][elem]["segmentation"]\n                    for elem in range(0, len(dataset_dict["annotations"]))]\n        input_to_aug = AugInput(image, \n                                boxes = bboxes,\n                                sem_seg = sem_segs)\n        to_return = augs(input_to_aug)\n        #resize_img_func = resize_obj.get_transform(image)\n        #dataset_dict["image"]

In [ ]:
from detectron2.data import detection_utils
from detectron2.data.transforms import ScaleTransform, AugInput, AugmentationList
from detectron2.data.transforms.augmentation_impl import ResizeShortestEdge

class SimpleResizeDatasetMapper:
    def __init__(self, cfg):
        self.cfg = cfg

    def __call__(self, dataset_dict):
      dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
      # can use other ways to read image
      image = detection_utils.read_image(dataset_dict["file_name"], format="RGB")
      # See "Data Augmentation" tutorial for details usage
      auginput = AugInput(image)
      resize_obj = ResizeShortestEdge(self.cfg.INPUT.MIN_SIZE_TEST,
                                          self.cfg.INPUT.MAX_SIZE_TEST,
                                          sample_style = "choice")

      transform = resize_obj(auginput)
      image = torch.from_numpy(auginput.image.transpose(2, 0, 1))
      annos = [
          detection_utils.transform_instance_annotations(annotation, [transform], image.shape[1:])
          for annotation in dataset_dict.pop("annotations")
      ]
      return {
        # create the format that the model expects
        "file_name": dataset_dict["file_name"],
        "height": image.shape[1],
        "width": image.shape[2],
        "image_id": dataset_dict["image_id"],
        "image": image,
        "instances": detection_utils.annotations_to_instances(annos, image.shape[1:],
                                                              mask_format = self.cfg.INPUT.MASK_FORMAT)
      }

In [ ]:
train_cfg = cfg_alt_dino.clone()
with open("/content/TACO-expl/MaskDINO/configs/taco/taco_train.yaml", "w") as f:
  f.write(train_cfg.dump())

In [ ]:
!mkdir /content/TACO-expl/maskdino_config/
!mkdir /content/output/
!mkdir /content/output/chkpt/
!cp /content/MyDrive/MyDrive/maskdino_models/model_0000478.pth /content/output/chkpt/

mkdir: cannot create directory ‘/content/TACO-expl/maskdino_config/’: File exists
mkdir: cannot create directory ‘/content/output/’: File exists
mkdir: cannot create directory ‘/content/output/chkpt/’: File exists


In [ ]:
train_cfg_loaded = get_cfg()
train_cfg_loaded.set_new_allowed(True)
train_cfg_loaded.merge_from_file("/content/TACO-expl/maskdino_config/taco_train_maskdino.yaml")
print(train_cfg_loaded.MODEL.BACKBONE.FREEZE_AT)
print(train_cfg_loaded.SOLVER.IMS_PER_BATCH)
print(train_cfg_loaded.INPUT.MIN_SIZE_TRAIN)
print(train_cfg_loaded.DATASETS.PRECOMPUTED_PROPOSAL_TOPK_TRAIN)
print(train_cfg_loaded.TEST.AUG.ENABLED)
print(train_cfg_loaded.INPUT.MASK_FORMAT)
print(train_cfg_loaded.Default_loading)

5
2
(480, 512, 544, 576, 608, 640, 672, 704, 736, 768, 800)
100
False
polygon
False


In [ ]:
val_mapper = SimpleResizeDatasetMapper(train_cfg_loaded)
#mapped_dicts_val = val_mapper(dataset_dicts_val[0])
#print(mapped_dicts_val)
from detectron2.data import DatasetMapper, build_detection_test_loader

val_dataloader = build_detection_test_loader(train_cfg_loaded, "TACO_val",
   mapper=val_mapper)

[05/15 16:50:42 d2.data.datasets.coco]: Loaded 150 images in COCO format from /content/TACO-expl/data/annotations_off_0_val.json
[05/15 16:50:42 d2.data.build]: Distribution of instances among all 10 categories:
|  category  | #instances   |   category    | #instances   |  category  | #instances   |
|:----------:|:-------------|:-------------:|:-------------|:----------:|:-------------|
|   Bottle   | 50           |  Bottle cap   | 30           |    Can     | 21           |
| Cigarette  | 42           |      Cup      | 21           |    Lid     | 8            |
|   Other    | 135          | Plastic bag.. | 77           |  Pop tab   | 10           |
|   Straw    | 16           |               |              |            |              |
|   total    | 410          |               |              |            |              |
[05/15 16:50:42 d2.data.common]: Serializing 150 elements to byte tensors and concatenating them all ...
[05/15 16:50:42 d2.data.common]: Serialized dataset takes 0.

In [ ]:
entry_cp = dataset_dicts_val[0].copy()
print(entry_cp.keys())
print(entry_cp["file_name"])
print(entry_cp["image_id"])
print(entry_cp["height"])
print(entry_cp["width"])
print(entry_cp["annotations"][0].keys())
print(entry_cp["annotations"][0]["bbox"])
print(entry_cp["annotations"][0]["category_id"])
#print(entry_cp["annotations"][0]["segmentation"][0])
to_print = val_mapper(entry_cp)

print(to_print.keys())
print(to_print["file_name"])
print(to_print["height"])
print(to_print["width"])
print(to_print["image_id"])
print(to_print["image"].shape)
print(to_print["instances"])

dict_keys(['file_name', 'height', 'width', 'image_id', 'annotations'])
/content/MyDrive/MyDrive/official/batch_1/000058.jpg
15
2049
1537
dict_keys(['iscrowd', 'bbox', 'category_id', 'segmentation', 'bbox_mode'])
[150.0, 328.0, 568.0, 1304.0]
0
dict_keys(['file_name', 'height', 'width', 'image_id', 'image', 'instances'])
/content/MyDrive/MyDrive/official/batch_1/000058.jpg
1066
800
15
torch.Size([3, 1066, 800])
Instances(num_instances=3, image_height=1066, image_width=800, fields=[gt_boxes: Boxes(tensor([[ 78.0742, 170.6432, 373.7150, 849.0542],
        [417.4366, 210.7028, 702.1470, 941.1391],
        [505.4001, 210.7028, 659.4665, 310.0713]])), gt_classes: tensor([0, 0, 1]), gt_masks: PolygonMasks(num_instances=3)])


<ipython-input-14-c70bda7d9e0b>:20: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  image = torch.from_numpy(auginput.image.transpose(2, 0, 1))


In [ ]:
from detectron2.data import DatasetMapper   # the default mapper
default_mapper = DatasetMapper(train_cfg_loaded, is_train=True)
default_mapped = default_mapper(entry_cp)
print(default_mapped.keys())
print(default_mapped["file_name"])
print(default_mapped["height"])
print(default_mapped["width"])
print(default_mapped["image_id"])
print(default_mapped["image"].shape)
print(default_mapped["instances"])

[05/15 16:50:44 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(480, 512, 544, 576, 608, 640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
dict_keys(['file_name', 'height', 'width', 'image_id', 'image', 'instances'])
/content/MyDrive/MyDrive/official/batch_1/000058.jpg
2049
1537
15
torch.Size([3, 768, 576])
Instances(num_instances=3, image_height=768, image_width=576, fields=[gt_boxes: Boxes(tensor([[ 56.2134, 122.9400, 269.0748, 611.7013],
        [300.5543, 151.8009, 505.5459, 678.0439],
        [363.8881, 151.8009, 474.8159, 223.3909]])), gt_classes: tensor([0, 0, 1])])


In [ ]:
%cd /content/TACO-expl/MaskDINO/
from train_net import Trainer, main

/content/TACO-expl/MaskDINO


In [ ]:
!python -m pip install numpy==1.23.1
import numpy as np
np.bool = np.bool_

In [ ]:
%cd /content/output/chkpt/

/content/output/chkpt


In [ ]:
trainer = Trainer(train_cfg_loaded)
#dir(trainer)

criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'loss_mask_interm_dn': 5.0, 'loss_dice_interm_dn': 5.0, 'loss_bbox_interm_dn': 5.0, 'loss_giou_interm_dn': 2.0, 'loss_ce_0': 4.0, 'loss_mask_0': 5.0, 'loss_dice_0': 5.0, 'loss_bbox_0': 5.0, 'loss_giou_0': 2.0, 'loss_ce_interm_0': 4.0, 'loss_mask_interm_0': 5.0, 'loss_dice_interm_0': 5.0, 'loss_bbox_interm_0': 5.0, 'loss_giou_interm_0': 2.0, 'loss_ce_dn_0': 4.0, 'loss_mask_dn_0': 5.0, 'loss_dice_dn_0': 5.0, 'loss_bbox_dn_0': 5.0, 'loss_giou_dn_0': 2.0, 'loss_ce_interm_dn_0': 4.0, 'loss_mask_interm_dn_0': 5.0, 'loss_dice_interm_dn_0': 5.0, 'loss_bbox_interm_dn_0': 5.0, 'loss_giou_interm_dn_0': 2.0, 'loss_ce_1': 4.0, 'loss_mask_1':

In [ ]:
#trainer.build_hooks()
trainer.resume_or_load(resume = True)  # load last checkpoint or MODEL.WEIGHTS
trainer.train()

[05/15 16:51:01 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                                         | Names in Checkpoint                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/15 16:51:22 d2.utils.events]:  eta: 3:04:25  iter: 2399  total_loss: 58.49  loss_ce: 1.612  loss_mask: 0.03262  loss_dice: 0.4572  loss_bbox: 0.2302  loss_giou: 1.083  loss_ce_dn: 0.4304  loss_mask_dn: 0.03233  loss_dice_dn: 0.6398  loss_bbox_dn: 0.05872  loss_giou_dn: 0.534  loss_ce_0: 1.606  loss_mask_0: 0.05097  loss_dice_0: 0.5688  loss_bbox_0: 0.3199  loss_giou_0: 1.215  loss_ce_dn_0: 0.7821  loss_mask_dn_0: 0.2558  loss_dice_dn_0: 2.467  loss_bbox_dn_0: 0.3566  loss_giou_dn_0: 0.8756  loss_ce_1: 1.837  loss_mask_1: 0.03535  loss_dice_1: 0.6906  loss_bbox_1: 0.2382  loss_giou_1: 0.9775  loss_ce_dn_1: 0.4471  loss_mask_dn_1: 0.04049  loss_dice_dn_1: 0.9481  loss_bbox_dn_1: 0.1509  loss_giou_dn_1: 0.5671  loss_ce_2: 1.525  loss_mask_2: 0.04509  loss_dice_2: 0.6527  loss_bbox_2: 0.2603  loss_giou_2: 0.9494  loss_ce_dn_2: 0.4048  loss_mask_dn_2: 0.03372  loss_dice_dn_2: 0.7199  loss_bbox_dn_2: 0.1258  loss_giou_dn_2: 0.5291  loss_ce_3: 1.593  loss_mask_3: 0.03139  loss_dice_3: 0.4

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


________THE IMAGE SIZE IS:________
torch.Size([1066, 800])
__________________________________
torch.Size([300, 10])
__________________________________
torch.Size([300, 4])
__________________________________
________THE IMAGE SIZE IS:________
torch.Size([1066, 800])
__________________________________
torch.Size([300, 10])
__________________________________
torch.Size([300, 4])
__________________________________
________THE IMAGE SIZE IS:________
torch.Size([1066, 800])
__________________________________
torch.Size([300, 10])
__________________________________
torch.Size([300, 4])
__________________________________
________THE IMAGE SIZE IS:________
torch.Size([800, 1066])
__________________________________
torch.Size([300, 10])
__________________________________
torch.Size([300, 4])
__________________________________
________THE IMAGE SIZE IS:________
torch.Size([1067, 800])
__________________________________
torch.Size([300, 10])
__________________________________
torch.Size([300, 4])


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[05/15 16:55:07 d2.engine.hooks]: Overall training speed: 102 iterations in 0:02:55 (1.7185 s / it)
[05/15 16:55:07 d2.engine.hooks]: Total training time: 0:03:54 (0:00:59 on hooks)
[05/15 16:55:07 d2.utils.events]:  eta: 1:23:37  iter: 2499  total_loss: 41.67  loss_ce: 0.9513  loss_mask: 0.04624  loss_dice: 0.548  loss_bbox: 0.06083  loss_giou: 0.3106  loss_ce_dn: 0.1839  loss_mask_dn: 0.03841  loss_dice_dn: 0.5354  loss_bbox_dn: 0.05288  loss_giou_dn: 0.2473  loss_ce_0: 1.426  loss_mask_0: 0.05391  loss_dice_0: 0.6026  loss_bbox_0: 0.05564  loss_giou_0: 0.335  loss_ce_dn_0: 0.7115  loss_mask_dn_0: 0.158  loss_dice_dn_0: 1.87  loss_bbox_dn_0: 0.2263  loss_giou_dn_0: 0.8563  loss_ce_1: 1.377  loss_mask_1: 0.03978  loss_dice_1: 0.5402  loss_bbox_1: 0.07733  loss_giou_1: 0.3491  loss_ce_dn_1: 0.3026  loss_mask_dn_1: 0.03921  loss_dice_dn_1: 0.5624  loss_bbox_dn_1: 0.0928  loss_giou_dn_1: 0.3437  loss_ce_2: 1.28  loss_mask_2: 0.03977  loss_dice_2: 0.5161  loss_bbox_2: 0.05184  loss_giou_2

KeyboardInterrupt: 

# Run this if you want to check how many trainable parameters we have

In [ ]:
from detectron2.modeling import build_model
model = build_model(train_cfg_loaded)
num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(num_trainable_params)

criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'loss_mask_interm_dn': 5.0, 'loss_dice_interm_dn': 5.0, 'loss_bbox_interm_dn': 5.0, 'loss_giou_interm_dn': 2.0, 'loss_ce_0': 4.0, 'loss_mask_0': 5.0, 'loss_dice_0': 5.0, 'loss_bbox_0': 5.0, 'loss_giou_0': 2.0, 'loss_ce_interm_0': 4.0, 'loss_mask_interm_0': 5.0, 'loss_dice_interm_0': 5.0, 'loss_bbox_interm_0': 5.0, 'loss_giou_interm_0': 2.0, 'loss_ce_dn_0': 4.0, 'loss_mask_dn_0': 5.0, 'loss_dice_dn_0': 5.0, 'loss_bbox_dn_0': 5.0, 'loss_giou_dn_0': 2.0, 'loss_ce_interm_dn_0': 4.0, 'loss_mask_interm_dn_0': 5.0, 'loss_dice_interm_dn_0': 5.0, 'loss_bbox_interm_dn_0': 5.0, 'loss_giou_interm_dn_0': 2.0, 'loss_ce_1': 4.0, 'loss_mask_1':